## 사용자 기반 협업 필터링 샘플

In [2]:
import pandas as pd
#rating; 평가
ratings_expand={
    '고길동': {
        '택시운전사': 3.5,
        '남한산성': 1.5,
        '킹스맨:골든서클': 3.0,
        '범죄도시': 3.5,
        '아이 캔 스피크': 2.5,
        '기생충': 3.0,
    },
    '도우너': {
        '택시운전사': 5.0,
        '남한산성': 4.5,
        '킹스맨:골든서클': 0.5,
        '범죄도시': 1.5,
        '아이 캔 스피크': 4.5,
        '기생충': 5.0,
    },
    '또치': {
        '택시운전사': 3.0,
        '남한산성': 2.5,
        '킹스맨:골든서클': 1.5,
        '범죄도시': 3.0,
        '기생충': 3.0,
        '아이 캔 스피크': 3.5,
    },
    '마이콜': {
        '택시운전사': 2.5,
        '남한산성': 3.0,
        '범죄도시': 4.5,
        '기생충': 4.0,
    },
    '듀크': {
        '남한산성': 4.5,
        '킹스맨:골든서클': 3.0,
        '기생충': 4.5,
        '범죄도시': 3.0,
        '아이 캔 스피크': 2.5,
    },
    '희동이': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '킹스맨:골든서클': 1.0,
        '범죄도시': 3.0,
        '기생충': 3.5,
        '아이 캔 스피크': 2.0,
    },
    '올라프': {
        '택시운전사': 3.0,
        '남한산성': 4.0,
        '기생충': 3.0,
        '범죄도시': 5.0,
        '아이 캔 스피크': 3.5,
    },
    '둘리': {
        '남한산성': 4.5, 
        '아이 캔 스피크': 1.0,
        '범죄도시': 4.0
    }
}
    
ratings = pd.DataFrame(ratings_expand).T
display(ratings)

,택시운전사,남한산성,킹스맨:골든서클,범죄도시,아이 캔 스피크,기생충
고길동,3.5,1.5,3.0,3.5,2.5,3.0
도우너,5.0,4.5,0.5,1.5,4.5,5.0
또치,3.0,2.5,1.5,3.0,3.5,3.0
마이콜,2.5,3.0,NaN,4.5,NaN,4.0
듀크,NaN,4.5,3.0,3.0,2.5,4.5
희동이,3.0,4.0,1.0,3.0,2.0,3.5
올라프,3.0,4.0,NaN,5.0,3.5,3.0
둘리,NaN,4.5,NaN,4.0,1.0,NaN


In [3]:
import math
import numpy as np

def similar_user(name, dataframe):
    # movies 안에는 name 사용자가 평가한 영화 이름만을 담는다.
    movies = []
    for i in dataframe.loc[name,:].index:
        if math.isnan(dataframe.loc[name,i]) == False:
            movies.append(i)
    
    # U_df는 name 사용자가 평가한 것만 추출합니다.
    U_df = pd.DataFrame(dataframe.loc[name,movies] ).T
    display(U_df)
    # other_df는 name 사용자를 제외한 데이터프레임입니다.
    other_df = dataframe.loc[:,movies].drop(name, axis=0)
    
  
    #sim_dict에 name과 다른 사용자 간의 유사도를 평가한 결과를 담는다.
    sim_dict = {}    
    
    # user와 name 사용자 둘 다 평점을 매긴 영화에 대한 벡터로 코사인 유사도를 구한다.
    for user in other_df.index:
        sm = [m for m in U_df.columns if math.isnan(other_df.loc[user,m])==False]        
        main_n = np.linalg.norm(U_df.loc[name,sm])
        user_n = np.linalg.norm(other_df.loc[user,sm])
        prod = np.dot(U_df.loc[name,sm], other_df.loc[user,sm])
        sim_dict[user] = prod/(main_n*user_n)      
    
    return sim_dict


similar_user('둘리', ratings)


,남한산성,범죄도시,아이 캔 스피크
둘리,4.5,4.0,1.0


{'고길동': 0.8362781852960594,
 '도우너': 0.7705732382498681,
 '또치': 0.8357844772998744,
 '마이콜': 0.9673722233802452,
 '듀크': 0.9556029765914315,
 '희동이': 0.973616310567801,
 '올라프': 0.931805248032154}

In [5]:
userlist=similar_user('둘리',ratings)

import operator
sorted(userlist.items() ,key=operator.itemgetter(1), reverse=True)[:2]

,남한산성,범죄도시,아이 캔 스피크
둘리,4.5,4.0,1.0


[('희동이', 0.973616310567801), ('마이콜', 0.9673722233802452)]

In [6]:
import math
import numpy as np

def recommand(name, dataframe):
    # movies 안에는 name 사용자가 평가한 영화 이름만을 담는다.
    movies = []
    for i in dataframe.loc[name,:].index:
        if math.isnan(dataframe.loc[name,i]) == False:
            movies.append(i)
    
    # U_df는 name 사용자가 평가한 것만 추출한다.
    U_df = pd.DataFrame(dataframe.loc[name,movies] ).T
    
    # other_df는 name 사용자를 제외한 데이터프레임이다.
    other_df = dataframe.loc[:,movies].drop(name, axis=0)   
    
    #sim_dict에 name과 다른 사용자 간의 유사도를 평가한 결과를 담는다.
    sim_dict = {}
    
    
    # user와 name 사용자 둘 다 평점을 매긴 영화에 대한 벡터로 코사인 유사도를 구한다.
    for user in other_df.index:
        sm = [m for m in U_df.columns if math.isnan(other_df.loc[user,m])==False]
        
        main_n = np.linalg.norm(U_df.loc[name,sm])
        user_n = np.linalg.norm(other_df.loc[user,sm])
        prod = np.dot(U_df.loc[name,sm], other_df.loc[user,sm])
        sim_dict[user] = prod/(main_n*user_n)
        
    ###########################################################################
    # 추천 영화 출력 #
    
    import operator
    sim_mat = sorted(sim_dict.items() ,key=operator.itemgetter(1), reverse=True)  
    
    # name 사용자가 평가하지 않은 영화를 추출합니다.
    recommend_list = list(set(dataframe.columns) - set(U_df.columns))
    others_k = [i[0] for i in sim_mat]
    recommender = dict()
    for movie in recommend_list:
        rating = []
        sim = []
        for person in others_k:
            if math.isnan(dataframe.loc[person,movie])==False:
                rating.append(dataframe.loc[person,movie])
                sim.append(sim_dict[person])
        pred = ((sim[0]*rating[0])+(sim[1]*rating[1]))/(sum(sim[:2]))
        recommender[movie]= pred
        
    return sorted(recommender.items(), key=operator.itemgetter(1), reverse=True)

In [7]:
recommand('둘리', ratings)

[('기생충', 3.749195759392914),
 ('택시운전사', 2.750804240607086),
 ('킹스맨:골든서클', 1.990662889337534)]

In [8]:
recommand('킹스맨:골든서클', ratings.T)

[('마이콜', 4.265722708205136),
 ('올라프', 4.062890832820543),
 ('둘리', 2.6551626147608633)]

### (참고) surprise 라는 추천 시스템 라이브러리도 있음